# Prediction investigation
To investigate why model predicts a certain class, we can look into the activations of the last convolutional layer. This might explain on which parts of the picture the model focuses and help us determine the reasoning for classification. Implementation is mostly based on Keras gradcam example (https://keras.io/examples/vision/grad_cam/). Highlighted regions (in red) are the regions that are most important for the final convolutional layer.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import tensorflow as tf
import keras

# Display
from IPython.display import Image, display
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
model = keras.models.load_model('../model/efficientnetb0.keras')

In [ ]:
img_size = (256, 256)
last_conv_layer_name = "top_activation"
labels = [
            "Facebook, Inc.",
            "Microsoft",
            "Microsoft OneDrive",
            "Orange",
            "Spotify",
            "Steam",
            "UPS",
            "Vodafone",
            "Wells Fargo & Company",
            "others",
        ]

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = keras.models.Model(
        model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:

def get_gradcam_img(img, heatmap, alpha=0.4):
    
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = mpl.colormaps["jet"]

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Display Grad CAM
    return superimposed_img

def create_grad_cam_image(img_array, model, last_conv_layer_name):
    
    # Print what the top predicted class is
    preds = model.predict(img_array)
    
    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    return get_gradcam_img(img_array[0], heatmap, alpha=0.4), labels[np.argmax(preds)]


In [ ]:
img_shape = (256, 256, 3)
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    directory='../data/train_data_dedup',
    labels='inferred',
    label_mode='categorical',
    batch_size=8,
    image_size=(img_shape[0], img_shape[1]),
    shuffle=True,
    validation_split=0.2,
    seed=235,
    subset="both",)

In [ ]:
for image, label in train_ds.take(1):
    for i in range(image.shape[0]):
        plt.figure(figsize=(20,20))
        ax = plt.subplot(2, 4, i+1)
        gc_img, pred_label = create_grad_cam_image(np.expand_dims(image[i], axis=0),  model, last_conv_layer_name)
        plt.imshow(gc_img)
        plt.title("Actual: {}, predicted: {}".format(train_ds.class_names[np.argmax(label[i])], pred_label))
        plt.axis("off")